In [3]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests

def register_face_to_server(server_url, client_id, face_encodings):
    data_to_send = {
        'client_id': client_id,
        'encodings': [encoding.tolist() for encoding in face_encodings]  # Converter arrays numpy para listas
    }

    try:
        response = requests.post(server_url, json=data_to_send)
        print("Resposta do servidor:", response.json())
        messagebox.showinfo("Cadastro", "Rosto cadastrado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar os dados para o servidor: {e}")
        messagebox.showerror("Erro", f"Erro ao cadastrar rosto: {e}")

def detect_faces_from_webcam_and_register_to_server(server_url):
    # Carregar o modelo pré-treinado do OpenCV DNN (Caffe model para detecção de rosto)
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    # Capturar a webcam
    video_capture = cv2.VideoCapture(0)

    # Verificar se a webcam foi acessada corretamente
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    print("Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)")

    stop = False
    face_encodings = []
    face_detected = False

    # Função para fechar a janela tkinter e liberar a câmera
    def on_close():
        nonlocal stop
        stop = True
        print("Fechando a janela e liberando a câmera...")
        
        if video_capture.isOpened():
            video_capture.release()
        
        root.destroy()

    # Função para salvar o cadastro
    def on_register():
        nonlocal face_encodings, face_detected
        client_id = entry_client_id.get()

        if not client_id:
            messagebox.showerror("Erro", "O Client ID não pode estar vazio.")
            return
        
        if face_encodings:
            register_face_to_server(server_url, client_id, face_encodings)
            face_detected = False  # Resetar para permitir nova captura
            reset_gui()
        else:
            messagebox.showerror("Erro", "Nenhum rosto detectado para cadastrar.")

    # Função para cancelar o registro
    def on_cancel():
        nonlocal face_detected
        face_detected = False  # Resetar para permitir nova captura
        reset_gui()

    # Função para resetar a interface
    def reset_gui():
        entry_client_id.delete(0, tk.END)  # Limpar o campo de Client ID
        btn_register.pack_forget()  # Ocultar o botão de cadastrar
        btn_cancel.pack_forget()  # Ocultar o botão de cancelar
        update_frame()  # Retomar a captura da webcam

    # Criação da janela tkinter
    root = tk.Tk()
    root.title("Cadastro de Rosto")

    # Criação de um label tkinter para exibir os frames da webcam
    label = tk.Label(root)
    label.pack()

    # Campo para o Client ID
    tk.Label(root, text="Client ID:").pack()
    entry_client_id = tk.Entry(root)
    entry_client_id.pack()

    # Botão para confirmar o cadastro
    btn_register = tk.Button(root, text="Cadastrar Rosto", command=on_register)

    # Botão para cancelar o cadastro
    btn_cancel = tk.Button(root, text="Cancelar", command=on_cancel)

    # Definir o atalho Ctrl+Q para interromper a captura e fechar a janela
    root.bind('<Control-q>', lambda event: on_close())

    # Fechar a janela ao clicar no botão de fechar
    root.protocol("WM_DELETE_WINDOW", on_close)

    def update_frame():
        nonlocal face_encodings, face_detected

        if stop or face_detected:
            return

        # Capturar frame por frame da webcam
        ret, frame = video_capture.read()

        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        # Pré-processar a imagem para a detecção de rosto
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        # Percorrer as detecções
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filtrar detecções fracas (threshold de confiança pode ser ajustado)
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                face_locations.append((startY, endX, endY, startX))

                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")

            face_encodings = face_recognition.face_encodings(frame, face_locations)

            # Pausar a captura e mostrar os botões de ação
            face_detected = True
            btn_register.pack()  # Mostrar o botão de cadastrar
            btn_cancel.pack()  # Mostrar o botão de cancelar
        else:
            label.after(10, update_frame)

    update_frame()
    root.mainloop()

# Testar a função com a URL de cadastro
server_url = 'http://127.0.0.1:5000/register'
detect_faces_from_webcam_and_register_to_server(server_url)


Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)
Fechando a janela e liberando a câmera...


Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests

def register_face_to_server(server_url, client_id, face_encodings):
    data_to_send = {
        'client_id': client_id,
        'encodings': [encoding.tolist() for encoding in face_encodings]  # Converter arrays numpy para listas
    }

    try:
        response = requests.post(server_url, json=data_to_send)
        print("Resposta do servidor:", response.json())
        messagebox.showinfo("Cadastro", "Rosto cadastrado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar os dados para o servidor: {e}")
        messagebox.showerror("Erro", f"Erro ao cadastrar rosto: {e}")

def detect_faces_from_webcam_and_register_to_server(server_url):
    # Carregar o modelo pré-treinado do OpenCV DNN (Caffe model para detecção de rosto)
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    # Capturar a webcam
    video_capture = cv2.VideoCapture(0)

    # Verificar se a webcam foi acessada corretamente
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    print("Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)")

    stop = False
    face_encodings = []

    # Função para fechar a janela tkinter e liberar a câmera
    def on_close():
        nonlocal stop
        stop = True
        print("Fechando a janela e liberando a câmera...")
        
        if video_capture.isOpened():
            video_capture.release()
        
        root.destroy()

    # Função para salvar o cadastro
    def on_register():
        nonlocal face_encodings
        client_id = entry_client_id.get()

        if not client_id:
            messagebox.showerror("Erro", "O Client ID não pode estar vazio.")
            return
        
        if face_encodings:
            register_face_to_server(server_url, client_id, face_encodings)
        else:
            messagebox.showerror("Erro", "Nenhum rosto detectado para cadastrar.")

    # Criação da janela tkinter
    root = tk.Tk()
    root.title("Cadastro de Rosto")

    # Criação de um label tkinter para exibir os frames da webcam
    label = tk.Label(root)
    label.pack()

    # Campo para o Client ID
    tk.Label(root, text="Client ID:").pack()
    entry_client_id = tk.Entry(root)
    entry_client_id.pack()

    # Botão para confirmar o cadastro
    btn_register = tk.Button(root, text="Cadastrar Rosto", command=on_register)
    btn_register.pack()

    # Definir o atalho Ctrl+Q para interromper a captura e fechar a janela
    root.bind('<Control-q>', lambda event: on_close())

    # Fechar a janela ao clicar no botão de fechar
    root.protocol("WM_DELETE_WINDOW", on_close)

    def update_frame():
        nonlocal face_encodings

        if stop:
            return

        # Capturar frame por frame da webcam
        ret, frame = video_capture.read()

        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        # Pré-processar a imagem para a detecção de rosto
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        # Percorrer as detecções
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filtrar detecções fracas (threshold de confiança pode ser ajustado)
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                face_locations.append((startY, endX, endY, startX))

                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")

            face_encodings = face_recognition.face_encodings(frame, face_locations)

            # Quando um rosto é detectado, para a captura por 3 segundos
            root.after(3000, update_frame)
        else:
            label.after(10, update_frame)

    update_frame()
    root.mainloop()

# Testar a função com a URL de cadastro
server_url = 'http://127.0.0.1:5000/register'
detect_faces_from_webcam_and_register_to_server(server_url)


Aguardando até que um rosto seja detectado... (Pressione Ctrl+Q para sair)
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Fechando a janela e liberando a câmera...
